# Modelling Cortico-Cerebellar Connectivity

## Questions:
* What is the best model of cortico-cerebellar connectivity? 
* What is the topography of connections between the cortex and the cerebellum?
    * One-to-one connection topography?

## Goals:
* Estimate a model on cortical activity patterns and predict cerebellar activity pattern on a new set of data.
* Find the most useful model of cortico-cerebellar connectivity

## Background:
### Cortico-cerebellar connectivity in non-human primates
* (Kelly & Strick, 2003)

### Cortico-cerebellar connectivity in humans
* Methods using resting state functional connectivity:
    * (Habas et al., 2009)
    * (Buckner et al., 2011)
    * (Marek et al., 2018)
* Methods using functional gradients:
    * (Tian et al., 2020)
    * (Guell et al., 2018)

### Gap:
* assuming one to one connections, the previous models of cortico-cerebellar connectivity in humans, have used a winner-take-all approach and discovered the functional organization of cerebellum. __How am I going to address this using PLS or bi-clustering?__
* cortico-cerebellar networks not assuming the one-to-one connection (with no assumptions at al)
    
### Hypothesis:
* relaxing the assumption of one-to-one connection, a connectivity model that incorporates the connection topography/organization between the two structures will perform better at prediction activity patterns of the cerebellum on new dataset. _might be good if I use some materials from introduction in the bi-clustering paper_

## Material and Methods:
* MDTB dataset:
    * two task sets, 4 sessions, 32 runs in total
* Cortical activity pattterns were used as predictors and cerebellar activity patterns were used as responses.

Re-iterating the __goal__ here: __estimate connectivity weights and connectivity pattern between the cerebellum and cortex that makes the best predictions for cerebellar activity__

** How can I investigate connection topography??

### Analysis:
Multivariate multiple regression will be used to model cerebellar activity profiles as a linear function of cortical activity profiles across a range of conditions! The regression model here is:
\begin{align*}
&Y_{n*m}\ = X_{n*p}W_{p*m} + E_{n*m}
\\
\end{align*}

Where $n$ is the number of conditions, $m$ is the number of cerebellar voxels, $p$ is the number of cortical voxels/tessels, $W$ is the matrix of connectivity weights, and $E$ is the matrix containing the residuals. $Y$ is the matrix where each column contains the activity profile of a voxel in the cerebellum (response matrix) and $X$ is the matrix where each column contains activity profile of a voxel in the neocortex (explanatory matrix, design matrix). 

### Limitations:
There is high multicollinearity between explanatory variables (activity profiles of cerebellar voxels) which results from correlations between activity profiles of cortical tessles and the fact that the number of observations (in this case the number of conditions) is lower than the number of explanatory variables. This multicollinearity makes the estimated regression coefficients highly variable and will lead to poor predictions.

### Regression models:
There are a number of approaches that try to overcome this limitation. These approaches are:
#### Step-wise Regression
Selecting a subset of explanatory variables through a stepwise regression. This subset selection method ultimately leads to a subset of explanatory variables that best explain the variance in the response variable. This method is time-consuming for the purpose of this analysis. However, it might be a good approach to find the overlaps between cerebellar targets of cortical regions. Basically, for each cerebellar voxel, we try to find the best set of explanatory variables (cortical tessels) that best predict the activity profile, create a map of prediction accuracy of models and investigate the overlap between different models (different subsets of tassels' activity profiles)
#### Ridge Regression
a shrinkage method that shrinks the regression coefficients by imposing a size penalty. 
* Ridge regression and PCA: 
    * Ridge regression shrinks the direction of small variances in the design matrix the most,
* Ridge regression as an optimization problem: It tries to optimize penalized sums of squares:
\begin{align*}
&J\ = \sum (y_i - W_0 + \sum x_{ij} W_j)^2 + \lambda \sum W_j^2
\\
\end{align*}
In matrix notation:
\begin{align*}
&J\ = (Y - XW)^T (Y - XW) + W^T W
\\
\end{align*}
    * $\lambda$ is a parameter that determines the amount of shrinkage of the parameters. The higher the , more shrinkage will be applied
    
#### Principal Component Regression (PCR)
This method tries to find a set of orthogonal variables that best explain the covariance between the explanatory variables by applying PCA to the design matrix X. Then it regresses Y on the set of these orthogonal variables. It is a two-step process and as a optimization problem: 
1. PCA to X and 
    * $X = ZV$ subject to constraint $VV^T = 1$
        - $Z$ contains score variables and $V$ is the matrix of weights that transforms $X$ into the latent space.
        - The objective function for this step is the objective function of PCA:
        \begin{align*}
        &J\ = |X - ZV^T|^2 = |X - XVV^T|^2
        \\
        \end{align*}
2. Regress $Y$ onto principal components of $X$
    * $Y = ZW + E$ with the objective function:
    \begin{align*}
    &J\ = |Y - ZW|^2 
    \\
    \end{align*}
    
Or we combine these steps together in a single step (this will give different results):
J = Y - ZW2 = Y - XVW2, subject to constraint: VTV= 1
\begin{align*}
&J\ = |Y - ZW|^2 = |Y - XVW|^2, \text{subject to constraint:}\ VV^T = 1
\\
\end{align*}

As the variables are orthogonal, the problem of multicollinearity is solved. However, like all the other methods that use PCA, a question remains: What is the best number of components? In addition to this, the components are chosen to maximize the explained variance of X, not Y. Hence, the chosen components might not be good candidates to explain the highest variance in Y. To overcome the later limitation of this method, partial least squares regression (PLS-R) is used.

#### Partial Least Squares Regression (PLS-R)
This method also tries to solve the problem of multicollinearity in the design matrix. It estimates the latent space for both $X$ and $Y$ so as to maximize the covariance between the two. Basically, it tries to project both $X$ and $Y$ onto latent spaces and it tries to maximize the covariance between the latent spaces of $X$ and $Y$.

\begin{align*}
&X_{n*p} = Z_{n*q} V_{p*q}^T + E_{n*p}
\\
\\
&Y_{n*m} = U_{n*q} Q_{m*q}^T + F_{n*m}
\\
\end{align*}
Where $n$ is the number of conditions, $m$ is the number of cerebellar voxels, $p$ is the number of cortical voxels/tessels and $q$ is the number of PLS components.
As an optimization problem:

* As an optimization problem :
    \begin{align*}
    &Z_{n*q} = X_{n*p}V_{p*q}, \text{subject to constraint:}\ VV^T = 1
    \\
    \end{align*}
    
    $V$ is the matrix of weights that transforms $X$ into a latent space ($Z$)
    
    \begin{align*}
    &U_{n*q} = Y_{n*m}C_{m*q}, \text{subject to constraint:}\ CC^T = 1
    \\
    \end{align*}
    
    $C$ is the matrix of weights that transforms $Y$ into a latent space ($U$)
    
    The objective is to find the latent variables (spaces) between which the covariance is maximized and at the same time the method tries to predict $Y$:
    
    \begin{align*}
    &cov(Z, U) = Z^TU = V^TX^TYC
    \\
    \end{align*}
    
    
* There are different implementations of the PLSR, each with different assumptions. There are both __iterative__ and __non-iterative__ methods. To my understanding the iterative method is the most common one. But there is also a non-iterative method. The difference between the results of iterative and non-iterative methods is in the __orthogonality__ of final latent variables. Briefly, using the iterative methods like __PLS1__ (for 1-D response variable), __PLS2__, and __SIMPLS__, the latent variables will be mutually orthogonal as these methods are __iterative__ and in the end of each iteration, the effect of each estimated latent variable is subtracted from the original matrices, a process called __deflation__. The deflation process guarantees mutual orthogonality of latent variables. There is, however, a non-iterative approach. The latent variables estimated using this method will not, in general, be mutually orthogonal. In any case, there is a eigenvalue problem. The iterative methods try to solve the eigenvalue problem iteratively and the non-iterative method tries to solve this eigenvalue problem in one go.
* The algorithm tries to find $T$ and $U$ __iteratively__ and then calculates the regression coefficients as $b_i= z_i^T u_i$ at each iteration of the algorithm. Eventually, we will have a diagonal matrix ($B$) where each element is calculated in each iteration. Finally:

    \begin{align*}
    &Y_{n*m} = X_{n*p}W_{pls}
    \\
    \end{align*}
    
    where $W_{pls}= V_{p*q}^T + B_{q*q} C_{q*m}^T$. Based on this formulation, $W_{pls}$ is a $p*m$ matrix
    
    
* _I need to clarify something here to better understand how the dimensions of the matrices are determined:_
    For $A_{p*q}$, The pseudoinverse of $A$ is defined as $A_{q*p}^+$. This is useful in determining the dimensions of matrix $V$ and $W_{pls}$.
    
* _Keep in mind that we have $V_{p*q}$, $V_{q*p}^T$, and based on the relationship between a matrix and its pseudoinverse: $V_{p*q}^{T+}$_ 
* $B_{q*q}$ is a diagonal matrix.
    * __Hypothesis__: _In this model, $B$ being diagonal implies that the one-to-one topography is valid. However, not restricting B to be diagonal will imply that the one-to-one topography does not necessarily hold. My hypothesis is that not restricting B to be a diagonal matrix, in other words not imposing the one-to-one connection topography, will yield better predictions. __What would this mean in terms of mathematical implementation of the algorithm to estimate the parameters and latent structures?___

#### Simultaneous Parameter Learning and Bi-clustering for Multi-Response Models (Yu et al., 2019)


## Results:

### Preliminary results:

In [13]:
# import packages needed to visualize the results
import numpy as np
import pandas as pd
import evaluation  # user-defined package
import os


import glob
import nibabel as nib

from nilearn import plotting
from nilearn import surface
from nilearn import datasets
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'svg' # other available formats are: 'retina', 'png', 'jpeg', 'pdf'
# plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

In [2]:
# To visualize the results:
# sestting defaults for some variables
returnSubjs = np.array([2,3,4,6,8,9,10,12,14,15,17,18,19,20,21,22,24,25,26,27,28,29,30,31])
# Setting different options of the functions
roi_dict1        = {'cortex':'tesselsWB162', 'cerebellum':'grey_nan'} 
roi_dict2        = {'cortex':'tesselsWB362', 'cerebellum':'grey_nan'} 
roi_dict3        = {'cortex':'tesselsWB642', 'cerebellum':'grey_nan'} 
trainExperiment  = 1
tMode            = 'crossed'

In [3]:
# Create a dataframe with all the evaluation info
DF = evaluation.eval_df(sn = returnSubjs, glm = 7, models = ['l2regress', 'plsregress'], 
                 rois = ['tesselsWB162', 'tesselsWB362', 'tesselsWB642'], 
                 testExper = 2)

# save the dataframe?

In [6]:
# interactive visualization of the results
@widgets.interact(model = ['plsregress', 'l2regress'],
                  xname = ['162', '362', '642'], 
                  yvar  = ['Rcv', 'Rnc', 'Rp'])
def plot_model_params(model, xname, yvar):
    # gets the section of the dataframe corresponding to the model
    dfx = DF.loc[DF.xname == 'tesselsWB%s'%xname]
    dfx_model = dfx.loc[dfx.model == model]
    
    # sort the dataframe based on values of params
    df2 = dfx_model.sort_values(by ='params' , ascending=False)
    
    # plot
    figure, axes = plt. subplots(nrows=1, ncols=2)
    plt.subplot(121)
    sns.barplot(x='model', y=yvar, hue='params', data=df2)
    plt.subplot(122)
    sns.lineplot(x='params', y=yvar, marker='o', err_style = 'band', 
                 dashes = False, data=df2)
    figure. tight_layout(pad=3.0)

interactive(children=(Dropdown(description='model', options=('plsregress', 'l2regress'), value='plsregress'), …

In [37]:
# Visualize PLS component loading maps for 7-component model
class Defaults: 
    BASE_DIR = '/Users/ladan/Documents/MATLAB/Projects/Connectivity/Connectivity_MDTB/python_code'
    CONN_DIR = os.path.join(BASE_DIR, 'sc1', 'connModels')
    NCOMP    = 7 # number of components
    SUIT_FUNCTIONAL_DIR = os.path.join(BASE_DIR, "preliminaryIMs")
    SUIT_ANATOMICAL_DIR = '/Users/ladan/Documents/MATLAB/suit/flatmap'
    
    
class VisualizeCerebellum:
    def __init__(self):
        self.contrast_type = "group_PLS"
        self.glm = "glm7" 
        self.vmax = 0.003
        self.surf_mesh = os.path.join(Defaults.SUIT_ANATOMICAL_DIR, "FLAT.surf.gii")

    def plot_surface(self, comp_numb):
        view = plotting.view_surf(surf_mesh = self.surf_mesh, 
                                surf_map    = self.surf_map, 
                                colorbar    = True,
                                vmax        = self.vmax,
                                title       = 'comp%d'%comp_numb) 
        view.open_in_browser()
   
    def visualize_loading_surface(self, comps_numb, num, xres):

        # get functional group dir

        # get all contrast images in gifti format
        fpath = os.path.join(Defaults.BASE_DIR,
                             Defaults.SUIT_FUNCTIONAL_DIR,
                             f"{self.contrast_type}{comps_numb}_{num}_{xres}.func.gii")

        # get surface mesh for SUIT
        self.surf_map = surface.load_surf_data(fpath).astype(int)
        self.surf_mesh = os.path.join(Defaults.SUIT_ANATOMICAL_DIR, "FLAT.surf.gii")

        self.plot_surface(num) 


# # example code to visualize group contrast(s) on flat map
vis = VisualizeCerebellum()
vis.visualize_loading_surface(7, 3, 162)